In [153]:
import warnings
warnings.filterwarnings('ignore')
import csv
import glob
import io
import numpy as np
import re

In [154]:
# Function to parse corpus files

def parse_corpus(fname, corpus):
    
    f = open(fname, 'r', encoding='utf-8')
    # File is non-csv format but comma "," does not exist.
    # Csv reader is used to avoid text garbling.
    iter_obj = csv.reader(f)
    lines = [v for v in iter_obj]
    f.close()
    
    text = ''
    for line in lines:
        if len(line) == 0:
            continue

        s = line[0]
        if len(s) == 0: continue

        # ％ｃｏｍ： is supplementary info
        if s[0:5] == "％ｃｏｍ：":
#                 print(' - Line', i, ': skip ％ｃｏｍ')
            continue

        if s[0] == '＠':
#                 print(' - Line', i, ': skip ＠ meta info ')
            continue
        else:
            #Replace stars with UNK
            s = s.replace('＊＊＊','UNK')
            #Replace speaker symbols with a separater
            if s[0] == 'F' or s[0] == 'M':
                s = 'SSSS'+s[5:]
            if s[0:2] == 'Ｘ：':
                s = 'SSSS'+s[2:]

            s = re.sub('F[0-9]{3}',"UNK",s)
            s = re.sub('M[0-9]{3}',"UNK",s)
            s = s.replace("＊","")


        # ?? Should be after concatinating the values ??
        for L, R in zip(['（', '＜', '【'], ['）', '＞', '】']):
            while s.find(L) != -1:
                left_pos = s.find(L)
                if s.find(R) != -1:
                    right_pos = s.find(R)
                    if left_pos > right_pos:
                        if s[0:4] == 'SSSS':
                            s = s.replace(s[4:right_pos+1], '', 1)
                        else:
                            s = s.replace(s[:right_pos+1], '', 1)
                    else:
                        s = s.replace(s[left_pos:right_pos+1], '')
                        if len(s) == 0:
                            continue
                else:
                    s=s[0:left_pos]

        if s != "\n" and s != "SSSS":
            text += s


    if text[0:4] != 'SSSS':
        text = 'SSSS' + text
    while text[0:4] == 'SSSS':
        next_pos = text[4:].find("SSSS")
        if next_pos == -1:
            corpus.append(text)
            break
        else:
            corpus.append(text[:4+next_pos])
            text = text[4+next_pos:]
        # Breaks up a long sentence
        # ?? The split lines are considered a talk between 2. Any influence in training ??
        if len(corpus[-1]) > 50:
            xs = corpus[-1].split('。')
            if len(xs) == 1:
                continue
            corpus.pop()
            if len(xs[0]) > 30:
                corpus.append(xs[0].split('、')[0] + '。')
            else:
                corpus.append(xs[0] + '。')
            while xs[-1] == '' or xs[-1] == ' ' or xs[-1] == '　':
                xs.pop()
            if len(xs) > 1:
                if len(xs[-1]) > 30:
                    corpus.append('SSSS' + xs[-1].split('、')[-1] + '。')
                else:
                    corpus.append('SSSS' + xs[-1] + '。')


In [121]:
# Parse corpus

print('...Started parsing corpus files\n')

fname_list = glob.glob('data/nucc/*')
print('The number of the corpus files are', len(fname_list), '\n')

corpus = []
for fname in fname_list:
    parse_corpus(fname, corpus)
print('The number of the corpus is', len(corpus), '\n')

with open('data/nucc_corpus.txt', 'w', encoding='utf-8') as f:
    for line in corpus:
        f.write(line + "\n")
print('...completed parsing corpus into "data/nucc_corpus.txt"\n')


...Started parsing corpus files

The number of the corpus files are 129 

The number of the corpus is 83076 

...completed parsing corpus into "data/nucc_corpus.txt"



In [98]:
# Morphological analysis

# !! Note this step can take more than minutes !!

print('...Started morphological analysis on "data/nucc_corpus.txt"\n')

!jumanpp -f < data/nucc_corpus.txt > data/nucc_corpus_analyzed.txt

print('...completed the analysis into "data/nucc_corpus_analyzed.txt"\n')
print('...extracting only tokens from "data/nucc_corpus.txt"\n')

!cat data/nucc_corpus_analyzed.txt | cut -f1 -d\  > data/nucc_tokens.txt

print('...completed the extracting into "data/nucc_tokens.txt"\n')

...Started morphological analysis on "data/nucc_corpus.txt"

...completed the analysis into "data/nucc_corpus_analyzed.txt"

...extracting only tokens from "data/nucc_corpus.txt"

...completed the extracting into "data/nucc_tokens.txt"



In [155]:
# Function to tokenize parsed corpus file

def tokenize_corpus(src_fname, dst_fname):
    
    f = open(src_fname, 'r')
    data = f.read()
    f.close()
    print('Input file is', src_fname.split('/')[-1], '\n')
    
    data = re.sub('.*SSSS.*', 'SSSS', data)
    data = re.sub('.*UNK.*', 'UNK', data)
    data = re.sub('@\n', '', data)
    data = re.sub('EOS\n', '', data)
    
#     with open(dst_fname, 'w') as f:
#         f.write(data)
    
    
    # File is non-csv format but comma "," does not exist.
    # Csv reader is used to avoid text garbling.
    iter_obj = csv.reader(io.StringIO(data))
    
    token_list = [v for v in iter_obj]
    print('The number of the input token is', len(token_list), '\n')
    
#     c = 0
#     for token in token_list:
#         if c >10: break
#         print(token)


In [ ]:
# Tokenize corpus

print('...Started tokenizing corpus file\n')

tokenize_corpus('data/nucc_tokens.txt', 'data/nucc_tokens.npy')

print('...completed tokenizing into "data/nucc_tokens.npy"\n')


...Started tokenizing corpus file

Input file is nucc_tokens.txt 

The number of the input token is 811177 

['SSSS']
['の']
['町']
['と']
['いう']
['の']
['は']
['ちい']
['ちゃく']
['って']
['。']
['SSSS']
['１']
['時間']
['かから']
['ない']
['ぐらい']
['だ']
['ね']
['。']
['４']
['、']
['５０']
['分']
['で']
['。']
['SSSS']
['そうそう']
['。']
['SSSS']
['散歩']
['を']
['して']
['る']
['ワン']
['ちゃん']
['に']
['会った']
['んだ']
['。']
['SSSS']
['城壁']
['の']
['上']
['を']
['やっぱ']
['観光']
['客']
['な']
['んだ']
['けど']
['ワン']
['ちゃん']
['連れて']
['きて']
['る']
['人']
['たち']
['結構']
['多くて']
['。']
['SSSS']
['で']
['。']
['SSSS']
['そう']
['したら']
['。']
['SSSS']
['おー']
['、']
['そら']
['地球']
['は']
['やっぱり']
['丸かった']
['みたいだ']
['ね']
['。']
['\u3000']
['SSSS']
['そう']
['したら']
['その']
['ワン']
['ちゃん']
['が']
['なんか']
['か']
['喜んじゃ']
['って']
['。']
['SSSS']
['なんか']
['すごーい']
['な']
['つかれちゃ']
['って']
['ね']
['。']
['SSSS']
['さっき']
['ね']
['、']
['別に']
['、']
['そんなに']
['なでて']
['も']
['い']
['ない']
['んだ']
['よ']
['。']
['SSSS']
['よしよし']
['って']
['言った']
['だけ']
['な']
['のに']
['。']
['SSSS']
['あら']
['ワン']


['もう']
['１']
['つ']
['簡易']
['ベッド']
['が']
['出せる']
['ような']
['部屋']
['な']
['んだ']
['けど']
['。']
['SSSS']
['あんた']
['たち']
['だけ']
['で']
['使って']
['いい']
['から']
['って']
['言って']
['。']
['SSSS']
['これ']
['が']
['ね']
['、']
['これ']
['が']
['１']
['日']
['目']
['の']
['チェスター']
['の']
['ホテル']
['。']
['その']
['汚い']
['。']
['SSSS']
['見かけ']
['は']
['きれいだ']
['けど']
['ここ']
['、']
['ここ']
['や']
['どっか']
['に']
['。']
['SSSS']
['見かけ']
['は']
['きれいだ']
['けど']
['。']
['SSSS']
['見かけ']
['は']
['きれいだった']
['わ']
['。']
['SSSS']
['うん']
['。']
['そう']
['そうそう']
['。']
['SSSS']
['ちょっと']
['ちょっと']
['こう']
['すきま']
['を']
['見る']
['と']
['ほこり']
['たまって']
['る']
['みたい']
['な']
['。']
['SSSS']
['の']
['ホテル']
['。']
['SSSS']
['これ']
['が']
['チェスター']
['の']
['カシードラル']
['の']
['中庭']
['。']
['SSSS']
['と']
['か']
['だったら']
['自分']
['たち']
['で']
['料理']
['すれば']
['いい']
['し']
['さあ']
['。']
['それ']
['は']
['キッチン']
['が']
['ついて']
['いる']
['わけで']
['は']
['ない']
['？']
['SSSS']
['じゃ']
['ない']
['。']
['SSSS']
['全然']
['。']
['それ']
['は']
['ＢＢ']
['と']
['いう']
['の']
['は']
['ベッド']
['・']
['アンド']
['・']
['ブレ

['？']
['\u3000']
['SSSS']
['それ']
['が']
['さあ']
['、']
['あの']
['さあ']
['、']
['ウィンダミヤ']
['の']
['さあ']
['、']
['レストラン']
['が']
['最低だった']
['。']
['SSSS']
['まずい']
['？']
['SSSS']
['だ']
['から']
['その']
['前']
['の']
['日']
['の']
['ジャケッツ']
['で']
['あたし']
['は']
['懲りて']
['いる']
['わけ']
['よ']
['。']
['SSSS']
['もう']
['今日']
['こそ']
['は']
['あったかい']
['もの']
['が']
['食べ']
['たい']
['と']
['。']
['SSSS']
['そう']
['。']
['で']
['、']
['パスタ']
['が']
['食べ']
['たい']
['と']
['か']
['って']
['思って']
['。']
['SSSS']
['で']
['。']
['SSSS']
['ちょっと']
['フラフラ']
['歩き']
['ながら']
['途中']
['で']
['いい']
['店']
['が']
['あったら']
['。']
['SSSS']
['あったら']
['そこ']
['に']
['入って']
['みよう']
['か']
['と']
['。']
['SSSS']
['入っちゃ']
['おう']
['か']
['と']
['思い']
['つつ']
['行った']
['じゃ']
['ん']
['。']
['SSSS']
['ポーク']
['。']
['SSSS']
['チキン']
['カレー']
['が']
['あった']
['の']
['。']
['SSSS']
['日本']
['の']
['カレー']
['と']
['も']
['違い']
['ます']
['よ']
['ね']
['。']
['SSSS']
['わあー']
['カレー']
['ライス']
['食べよう']
['と']
['思って']
['、']
['あったかい']
['し']
['と']
['思って']
['。']
['SSSS']
['ご飯']
['が']
['食べ']
['れる']
['と']
['か']

['そんな']
['の']
['流れて']
['た']
['。']
['SSSS']
['最初']
['から']
['。']
['SSSS']
['最初']
['から']
['？']
['SSSS']
['だ']
['から']
['。']
['SSSS']
['このまま']
['の']
['状態']
['だった']
['もん']
['で']
['いっちゃ']
['った']
['じゃ']
['ん']
['。']
['SSSS']
['あー']
['、']
['それ']
['で']
['行って']
['る']
['間']
['に']
['。']
['SSSS']
['ほい']
['で']
['行って']
['る']
['間']
['に']
['。']
['SSSS']
['あー']
['、']
['落ちた']
['、']
['落ちた']
['。']
['SSSS']
['あたし']
['。']
['SSSS']
['飛行機']
['が']
['落ちたー']
['と']
['か']
['と']
['言って']
['。']
['SSSS']
['何']
['か']
['あった']
['よ']
['って']
['言って']
['、']
['アクシデント']
['だ']
['と']
['か']
['な']
['んだ']
['か']
['と']
['か']
['って']
['言って']
['る']
['けど']
['って']
['言って']
['。']
['SSSS']
['で']
['。']
['SSSS']
['知ら']
['ん']
['かった']
['じゃ']
['ん']
['ね']
['。']
['SSSS']
['まだまだ']
['だ']
['わ']
['って']
['。']
['SSSS']
['知って']
['るっ']
['と']
['か']
['言わ']
['れて']
['、']
['はっと']
['か']
['って']
['言って']
['。']
['SSSS']
['知ら']
['ない']
['よ']
['って']
['。']
['何']
['が']
['と']
['か']
['言って']
['。']
['SSSS']
['ねー']
['。']
['SSSS']
['明らかに']
['。']
['SSSS']
['で']
['も']
['それ']
['って']

['よ']
['ね']
['。']
['SSSS']
['ビンラディン']
['が']
['出て']
['こ']
['ない']
['もん']
['。']
['SSSS']
['どう']
['UNK']
['して']
['も']
['さあ']
['、']
['ほか']
['の']
['もの']
['が']
['さあ']
['。']
['SSSS']
['やめ']
['ない']
['と']
['言って']
['た']
['もん']
['ね']
['。']
['SSSS']
['最終']
['目標']
['は']
['ビンラディン']
['だ']
['から']
['さあ']
['。']
['SSSS']
['どう']
['するー']
['。']
['SSSS']
['ビンラディン']
['とっ']
['つかまえる']
['まで']
['は']
['きっと']
['UNK']
['。']
['SSSS']
['なんで']
['でも']
['証拠']
['を']
['見せ']
['ない']
['の']
['。']
['SSSS']
['証拠']
['は']
['ある']
['と']
['か']
['言って']
['た']
['よ']
['ねー']
['。']
['SSSS']
['言って']
['て']
['見せる']
['って']
['言って']
['て']
['。']
['SSSS']
['明らかな']
['の']
['は']
['ある']
['と']
['か']
['って']
['言って']
['た']
['よ']
['ね']
['。']
['SSSS']
['で']
['、']
['ペーパー']
['で']
['見せる']
['って']
['言って']
['、']
['それ']
['は']
['見て']
['ない']
['よ']
['ね']
['。']
['SSSS']
['見せて']
['ない']
['もの']
['。']
['何で']
['見せ']
['ない']
['の']
['。']
['SSSS']
['ない']
['んじゃ']
['ない']
['の']
['。']
['実は']
['。']
['SSSS']
['うーん']
['。']
['SSSS']
['あの']
['さあ']
['、']
['そう']
['、']
['そう']
['、']
['そう']


...

['で']
['ない']
['色']
['って']
['ちょっと']
['怖い']
['です']
['ね']
['。']
['SSSS']
['怖い']
['よ']
['ねー']
['。']
['SSSS']
['イカ']
['すみ']
['の']
['スパゲティ']
['と']
['か']
['、']
['最初']
['見た']
['とき']
['びっくり']
['した']
['。']
['SSSS']
['アメリカ']
['で']
['さー']
['、']
['紫']
['と']
['さー']
['、']
['緑']
['の']
['ケチャップ']
['が']
['出た']
['の']
['知って']
['る']
['？']
['SSSS']
['何']
['から']
['作る']
['んです']
['か']
['。']
['SSSS']
['たぶん']
['着色']
['料']
['が']
['混ざって']
['る']
['。']
['SSSS']
['はー']
['。']
['SSSS']
['うん']
['、']
['たぶん']
['。']
['このまま']
['八']
['事']
['に']
['行ける']
['？']
['出']
['れる']
['？']
['SSSS']
['行け']
['ます']
['、']
['行け']
['ます']
['。']
['SSSS']
['じゃあ']
['、']
['そっち']
['の']
['方']
['から']
['行った']
['方']
['が']
['。']
['SSSS']
['あのー']
['、']
['ジャスコ']
['の']
['下']
['。']
['SSSS']
['それ']
['を']
['あがっ']
['、']
['あ']
['、']
['それ']
['を']
['右']
['にー']
['。']
['SSSS']
['あっ']
['、']
['ケーキ']
['屋']
['さん']
['。']
['SSSS']
['そう']
['、']
['そう']
['、']
['うん']
['、']
['じゃあ']
['、']
['右']
['に']
['曲がって']
['もらって']
['。']
['SSSS']
['雨']
['が']
['降って']
['きちゃ']
['った']
['よ']
['。']

['。']
['SSSS']
['車']
['は']
['いい']
['ねー']
['。']
['やっぱり']
['ね']
['。']
['SSSS']
['うーん']
['、']
['いい']
['です']
['ねー']
['。']
['車']
['って']
['でも']
['維持']
['費']
['かかる']
['よ']
['ね']
['。']
['SSSS']
['かかる']
['？']
['SSSS']
['うーん']
['、']
['駐車']
['場']
['代']
['が']
['。']
['SSSS']
['月']
['に']
['それ']
['を']
['２']
['回']
['やった']
['と']
['したら']
['。']
['SSSS']
['あー']
['、']
['６０００']
['円']
['ねー']
['。']
['SSSS']
['で']
['、']
['駐車']
['場']
['代']
['が']
['７０００']
['円']
['。']
['SSSS']
['何で']
['そんな']
['ちょくちょく']
['帰る']
['の']
['？']
['SSSS']
['何']
['か']
['別に']
['、']
['何']
['か']
['ある']
['よー']
['と']
['か']
['言う']
['と']
['すぐ']
['に']
['。']
['SSSS']
['近い']
['もん']
['ね']
['。']
['SSSS']
['で']
['、']
['その']
['バイト']
['、']
['バイト']
['やん']
['ない']
['と']
['か']
['で']
['。']
['SSSS']
['バイト']
['に']
['帰る']
['の']
['？']
['SSSS']
['えーっ']
['、']
['それ']
['だ']
['と']
['UNK']
['ちゃん']
['の']
['こと']
['何も']
['言え']
['んじゃ']
['ん']
['。']
['SSSS']
['あっ']
['、']
['そっ']
['か']
['。']
['SSSS']
['と']
['、']
['家']
['帰る']
['ついで']
['と']
['か']
['、']
['友達']
['が']
['集まる']
['って'

['通って']
['らっしゃる']
['んです']
['か']
['。']
['SSSS']
['通って']
['る']
['って']
['いう']
['か']
['。']
['SSSS']
['適当に']
['アレンジ']
['できる']
['んです']
['けど']
['。']
['SSSS']
['先生']
['、']
['授業']
['は']
['どの']
['、']
['月']
['、']
['月']
['、']
['火']
['、']
['水']
['、']
['木']
['。']
['SSSS']
['水']
['、']
['木']
['、']
['金']
['。']
['SSSS']
['水']
['、']
['木']
['、']
['金']
['。']
['SSSS']
['でも']
['、']
['来']
['学期']
['も']
['なんか']
['、']
['月']
['、']
['水']
['、']
['金']
['か']
['、']
['月']
['、']
['木']
['、']
['金']
['か']
['。']
['SSSS']
['英語']
['は']
['あれ']
['です']
['よ']
['ね']
['。']
['教えて']
['らっしゃる']
['んです']
['よ']
['ね']
['。']
['大変です']
['よ']
['ね']
['。']
['SSSS']
['先生']
['は']
['何']
['を']
['教えて']
['いらっしゃる']
['んです']
['か']
['。']
['SSSS']
['フランス']
['語']
['です']
['。']
['SSSS']
['あっ']
['、']
['フランス']
['語']
['の']
['方']
['なんか']
['もっと']
['大変']
['。']
['SSSS']
['いえ']
['、']
['そんな']
['もっと']
['大変で']
['なくて']
['。']
['SSSS']
['それ']
['を']
['こう']
['しよう']
['と']
['思って']
['生きて']
['きた']
['わけじゃ']
['ない']
['んです']
['よ']
['ね']
['。']
['SSSS']
['はい']
['、']
['違う']
['んです']
['。

['強い']
['。']
['SSSS']
['うーん']
['、']
['知ら']
['ん']
['けどー']
['、']
['なんかー']
['。']
['それ']
['は']
['あれ']
['です']
['か']
['。']
['どんな']
['とこ']
['で']
['教え']
['てらした']
['んです']
['か']
['。']
['SSSS']
['えっとー']
['。']
['SSSS']
['なんか']
['そういう']
['感じ']
['です']
['。']
['SSSS']
['日本']
['語']
['の']
['先生']
['と']
['して']
['働いて']
['。']
['SSSS']
['はい']
['。']
['何']
['年間']
['か']
['、']
['はい']
['、']
['働いて']
['ました']
['。']
['SSSS']
['ふん']
['、']
['ふん']
['。']
['それ']
['、']
['大学']
['で']
['。']
['SSSS']
['大学']
['で']
['、']
['はい']
['。']
['だから']
['全然']
['違う']
['んです']
['ね']
['、']
['こっち']
['の']
['大学生']
['と']
['。']
['はい']
['。']
['SSSS']
['違い']
['ます']
['よ']
['ね']
['、']
['それ']
['は']
['確かに']
['。']
['SSSS']
['。']
['SSSS']
['そう']
['、']
['訓練']
['です']
['よ']
['ね']
['。']
['きっと']
['。']
['SSSS']
['訓練']
['、']
['うん']
['、']
['習って']
['ない']
['から']
['ね']
['。']
['SSSS']
['うーん']
['。']
['SSSS']
['でも']
['仕事']
['が']
['増えた']
['な']
['と']
['思って']
['。']
['SSSS']
['そや']
['けど']
['、']
['ゆう']
['て']
['も']
['まだ']
['早い']
['。']
['まだ']
['早い']
['って']
['、']
['まだ']
['、']
[

['。']
['SSSS']
['いえ']
['。']
['SSSS']
['ちょうど']
['いい']
['なって']
['いう']
['感じ']
['です']
['よ']
['ね']
['。']
['SSSS']
['まあ']
['、']
['ここ']
['でも']
['忙しい']
['です']
['よ']
['ね']
['。']
['SSSS']
['もう']
['いっそがしい']
['ところ']
['で']
['。']
['SSSS']
['何']
['か']
['事務']
['的な']
['こと']
['が']
['たくさん']
['ある']
['んです']
['よ']
['ね']
['。']
['SSSS']
['事務']
['的な']
['こと']
['も']
['いっぱい']
['ある']
['けど']
['。']
['SSSS']
['楽しく']
['やって']
['ます']
['。']
['SSSS']
['みんな']
['、']
['いい']
['人']
['です']
['ね']
['、']
['ここ']
['。']
['SSSS']
['いい']
['人']
['でしょう']
['。']
['SSSS']
['うん']
['。']
['SSSS']
['別']
['に']
['悪い']
['イメージ']
['じゃ']
['なかった']
['んです']
['けど']
['、']
['どー']
['しよう']
['と']
['思って']
['。']
['SSSS']
['うん']
['、']
['心配だった']
['けどー']
['って']
['いう']
['こと']
['で']
['ね']
['。']
['SSSS']
['はい']
['。']
['SSSS']
['事務']
['の']
['人']
['も']
['いい']
['人']
['だ']
['し']
['。']
['SSSS']
['ええ']
['、']
['ほんとに']
['もう']
['。']
['SSSS']
['はい']
['。']
['SSSS']
['そう']
['です']
['か']
['。']
['SSSS']
['んで']
['。']
['SSSS']
['偉い']
['ね']
['。']
['出し']
['たん']
['。']
['SSSS']
['もう']
['

In [101]:
iter_obj = csv.reader(open('data/nucc_corpus_analyzed.txt', 'r'), delimiter=' ')


In [102]:
li = [v[0] for v in iter_obj] 

In [80]:
mat = [v[0] for v in li] 

In [103]:
len(li)

935512